In [1]:
# Data
import numpy as np
import awkward as ak
import pandas as pd

# I/O
import uproot as ur
import hipopy.hipopy as hp # <--- Package for reading in the hipo files

# Plotting
import matplotlib.pyplot as plt

# Physics
from particle import PDGID

# Miscellaneous
import os
import sys #NOTE: ADDED
import tqdm


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-7ezchsmc because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
file_list = [
     '/volatile/clas12/users/mfmce/mc_jobs_rga_vtx_3_23_23/skim_50nA_OB_job_3313_0.hipo'
]
banks = [
    'REC::Particle',
    'MC::Lund',
    'REC::Traj',
    'REC::Track',
]
step  = 1000

def get_bank_keys(bank_name,all_keys,separator='_'):
    """
    :description: Get list of the keys for given bank name from a list of all batch keys.
    
    :param: bank_name
    :param: all_keys
    :param: separator='_'
    
    :return: bank_keys
    """
    bank_keys = []
    for key in all_keys:
        if key.startswith(bank_name+separator):
            bank_keys.append(key)
    return bank_keys
        
def get_event_table(bank_keys,event_num,batch,dtype=float):
    """
    :description: Get a bank event table as a numpy array of shape (number of columns, number of rows).
    
    :param: bank_keys
    :param: event_num
    :param: batch
    :param: dtype=float
    
    :return: bank_table
    """
    bank_table = []
    bank_table = np.moveaxis(np.array([batch[key][event_num] for key in bank_keys], dtype=dtype),[0,1],[1,0])
    return bank_table

def get_link_indices(event_table_rec_particle,event_table,pindex_idx=1):
    """
    :description: Get index pairs linking entries in a bank back to entries in the 'REC::Particle' bank.
    
    :param: event_table_rec_particle
    :param: event_table
    :param: pindex_idx=1
    
    :return: link_indices
    """
    
    link_indices = []
    nrec = np.shape(event_table_rec_particle)[0]
    for rec_particle_idx in range(0,nrec):
        for event_table_idx, el in enumerate(event_table[:,pindex_idx]):
            if el==rec_particle_idx:
                link_indices.append([event_table_idx,rec_particle_idx])
    return np.array(link_indices) #NOTE: link_indices = [(event_table_idx,rec_particle_idx)]

def get_parent_indices(mc_event_table,index_idx=0,parent_idx=4,daughter_idx=5):
    """
    TODO
    """
    for mc_event_table_idx, index in enumerate(mc_event_table[:,index_idx]):
        pass
    pass

def get_match_indices(
    rec_event_table,
    mc_event_table,
    rec_px_idx             = 0,
    rec_py_idx             = 1,
    rec_pz_idx             = 2,
    rec_ch_idx             = 6,
    mc_px_idx              = 6,
    mc_py_idx              = 7,
    mc_pz_idx              = 8,
    mc_pid_idx             = 3,
    mc_daughter_idx        = 5,
    match_charge           = True,
    require_no_mc_daughter = True,
    enforce_uniqueness     = True,
    ):
    """
    :description: Get index pairs matching 
    
    :param: rec_event_table
    :param: mc_event_table
    :param: rec_px_idx             = 0,
    :param: rec_py_idx             = 1,
    :param: rec_pz_idx             = 2,
    :param: rec_ch_idx             = 6,
    :param: mc_px_idx              = 6,
    :param: mc_py_idx              = 7,
    :param: mc_pz_idx              = 8,
    :param: mc_pid_idx             = 3,
    :param: mc_daughter_idx        = 5,
    :param: match_charge           = True,
    :param: require_no_mc_daughter = True,
    :param: enforce_uniqueness     = True,
    
    :return: match_indices
    """
    
    # Set minimum
    rec_final_state_min_idx = 1
    mc_final_state_min_idx  = 3 #NOTE: MC::Lund bank is structured [e, p, q, e', all the other final state particles...]
    
    # Initialize index map
    match_indices    = -np.ones((rec_event_table.shape[0],2),dtype=float)
    match_indices[0] = [0,3] #NOTE: Always match first entry in REC::Particle to scattered electron in MC::Lund.
    
    # Get REC::Particle info
    rec_pz    = rec_event_table[:,rec_pz_idx]
    rec_pT    = np.sqrt(np.square(rec_event_table[:,rec_px_idx])+np.square(rec_event_table[:,rec_py_idx]))
    rec_p     = np.sqrt(np.square(rec_event_table[:,rec_px_idx])+np.square(rec_event_table[:,rec_py_idx])+np.square(rec_event_table[:,rec_pz_idx]))
    rec_theta = np.arctan(rec_pT,rec_pz)
    offset    = np.sign(rec_event_table[:,rec_py_idx]) * np.array([np.pi/2 if el<0.0 else 0.0 for el in rec_event_table[:,rec_px_idx]])
    rec_phi   = offset + np.arctan(rec_event_table[:,rec_py_idx],rec_event_table[:,rec_px_idx]) #NOTE: THIS REQUIRES type(rec_event_table[:,rec_p*_idx])==np.array
    
    # Get MC::Lund info
    mc_pz    = mc_event_table[:,mc_pz_idx]
    mc_pT    = np.sqrt(np.square(mc_event_table[:,mc_px_idx])+np.square(mc_event_table[:,mc_py_idx]))
    mc_p     = np.sqrt(np.square(mc_event_table[:,mc_px_idx])+np.square(mc_event_table[:,mc_py_idx])+np.square(mc_event_table[:,mc_pz_idx]))
    mc_theta = np.arctan(mc_pT,mc_pz)
    offset   = np.sign(mc_event_table[:,mc_py_idx]) * np.array([np.pi/2 if el<0.0 else 0.0 for el in mc_event_table[:,mc_px_idx]])
    mc_phi   = offset + np.arctan(mc_event_table[:,mc_py_idx],mc_event_table[:,mc_px_idx]) #NOTE: THIS REQUIRES type(mc_event_table[:,mc_p*_idx])==np.array

    # Loop rec particles
    for rec_idx, rec_p in enumerate(rec_event_table):
        
        # Start with final state particles past scattered electron
        if rec_idx<rec_final_state_min_idx: continue
        
        # Get REC::Particle charge
        rec_ch = rec_event_table[rec_idx,ch_idx]
        
        # Loop mc particles
        mc_match_idx = -1
        min_domega   = 9999
        for mc_idx, mc_p in enumerate(mc_event_table):
            
            # Start with final state particles past scattered electron
            if mc_idx<mc_final_state_min_idx: continue
            
            # Enforce unique matching
            if enforce_uniqueness and mc_idx in match_indices[:,2]: continue
            
            # Match charge and require that the MC particle be final state (no daughters)
            if match_charge and rec_ch!=PDGID(mc_event_table[mc_idx,mc_pid_idx]).charge: continue
            if require_no_mc_daughter and mc_event_table[mc_idx,mc_daughter_idx]!=0: continue
                
            # Get angular and momentum differences
            dp     = np.abs(rec_p[rec_idx]     - mc_p[mc_idx])
            dtheta = np.abs(rec_theta[rec_idx] - mc_theta[mc_idx])
            dphi   = np.abs(rec_phi[rec_idx]   - mc_phi[mc_idx]) if np.abs(rec_phi[rec_idx] - mc_phi[mc_idx])<np.pi else 2*np.pi-np.abs(rec_phi[rec_idx]   - mc_phi[mc_idx])
            domega = dp**2 + dtheta**2 + dphi**2
            
            # Reset angular, momentum minimum difference
            if domega<min_domega:
                min_domega   = domega
                mc_match_idx = mc_idx
                
        # Append matched index pair
        match_indices[rec_idx] = [rec_idx,mc_match_idx]
        
    return np.array(match_indices)

In [4]:
# Iterate hipo file
for i, batch in tqdm.tqdm(enumerate(hp.iterate(file_list,banks=banks,step=step))):
    
    # Loop events in batch
    for event_num, _ in enumerate(range(0,len(batch[list(batch.keys())[0]]))):
        
        bank_name   = 'REC::Particle'
        print("DEBUGGING: bank_name   = ",bank_name)
        all_keys = list(batch.keys())
        print("DEBUGGING: all_keys    = ",all_keys)
        bank_keys   = get_bank_keys(bank_name,all_keys)
        print("DEBUGGING: bank_keys   = ",bank_keys)
        event_table_rec_particle = get_event_table(bank_keys,event_num,batch,dtype=float)
        print("DEBUGGING: event_table = ",event_table_rec_particle)
        print("DEBUGGING: np.shape(event_table) = ",np.shape(event_table_rec_particle))
        
        bank_name   = 'REC::Traj'
        print("DEBUGGING: bank_name2   = ",bank_name)
        all_keys = list(batch.keys())
        print("DEBUGGING: all_keys2    = ",all_keys)
        bank_keys   = get_bank_keys(bank_name,all_keys)
        print("DEBUGGING: bank_keys   = ",bank_keys)
        event_table = get_event_table(bank_keys,event_num,batch,dtype=float)
        print("DEBUGGING: event_table = ",event_table)
        print("DEBUGGING: np.shape(event_table) = ",np.shape(event_table))
        
        link_indices = get_link_indices(event_table_rec_particle,event_table,pindex_idx=1)
        print("DEBUGGING: links_indices = ",link_indices)
        
        bank_name   = 'MC::Lund'
        print("DEBUGGING: bank_name   = ",bank_name)
        all_keys = list(batch.keys())
        print("DEBUGGING: all_keys    = ",all_keys)
        bank_keys   = get_bank_keys(bank_name,all_keys)
        print("DEBUGGING: bank_keys   = ",bank_keys)
        mc_event_table = get_event_table(bank_keys,event_num,batch,dtype=float)
        print("DEBUGGING: mc_event_table = ",mc_event_table)
        print("DEBUGGING: np.shape(mc_event_table) = ",np.shape(mc_event_table))
        
        # Rename REC::Particle event table
        rec_event_table = event_table_rec_particle
        
        # Get MC->REC Match indices
        match_indices = get_match_indices(rec_event_table,mc_event_table)
        print("DEBUGGING: match_indices = ",match_indices)
        
        
#         x = torch.moveaxis(torch.tensor([batch[key][0] for key in data_keys], dtype=torch.float),[0,1],[1,0])
        
#         # Get arrays for REC::Particle
#         px = batch['REC::Particle_px'][event_num]
#         py = batch['REC::Particle_py'][event_num]
#         pz = batch['REC::Particle_pz'][event_num]
#         vx = batch['REC::Particle_vx'][event_num]
#         vy = batch['REC::Particle_vy'][event_num]
#         vz = batch['REC::Particle_vz'][event_num]
#         c2 = batch['REC::Particle_chi2pid'][event_num]
#         bt = batch['REC::Particle_beta'][event_num]
#         st = batch['REC::Particle_status'][event_num]
        
#         # Get pion indices in REC::Particle
#         for pindex, px in enumerate(batch['REC::Particle_pid'][event_num]):
#             print(pindex,px)
#             break
        
#         # Find MC matches
#         for pindex, px in enumerate(batch['REC::Particle_px'][event_num]):
#             print(pindex,px)
#             break
            
        # Append to output array a list of [(pion_rec, part_true), (additional input array for NN) for pion_rec in event]
        break
    
    # Add output
    break

0it [00:00, ?it/s]

DEBUGGING: bank_name   =  REC::Particle
DEBUGGING: all_keys    =  ['REC::Particle_pid', 'REC::Particle_px', 'REC::Particle_py', 'REC::Particle_pz', 'REC::Particle_vx', 'REC::Particle_vy', 'REC::Particle_vz', 'REC::Particle_vt', 'REC::Particle_charge', 'REC::Particle_beta', 'REC::Particle_chi2pid', 'REC::Particle_status', 'MC::Lund_index', 'MC::Lund_lifetime', 'MC::Lund_type', 'MC::Lund_pid', 'MC::Lund_parent', 'MC::Lund_daughter', 'MC::Lund_px', 'MC::Lund_py', 'MC::Lund_pz', 'MC::Lund_energy', 'MC::Lund_mass', 'MC::Lund_vx', 'MC::Lund_vy', 'MC::Lund_vz', 'REC::Traj_pindex', 'REC::Traj_index', 'REC::Traj_detector', 'REC::Traj_layer', 'REC::Traj_x', 'REC::Traj_y', 'REC::Traj_z', 'REC::Traj_cx', 'REC::Traj_cy', 'REC::Traj_cz', 'REC::Traj_path', 'REC::Track_index', 'REC::Track_pindex', 'REC::Track_detector', 'REC::Track_sector', 'REC::Track_status', 'REC::Track_q', 'REC::Track_chi2', 'REC::Track_NDF']
DEBUGGING: bank_keys   =  ['REC::Particle_pid', 'REC::Particle_px', 'REC::Particle_py', '

TypeError: Cannot interpret '2' as a data type